In [1]:
!nvidia-smi

Wed Apr 13 06:04:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 6.5 MB 57.1 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [5]:
import json
content={"username":"namlv1997","key":"5ecc995bb891689ee7f58417d65cbe37"}
with open('kaggle.json','w')as f:
  json.dump(content,f)

In [6]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c fake-news

 75% 35.0M/46.5M [00:00<00:00, 128MB/s]
100% 46.5M/46.5M [00:00<00:00, 101MB/s]


In [8]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [14]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from sklearn.metrics import f1_score
from transformers import AutoConfig
from sklearn.model_selection import train_test_split

In [74]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"

In [75]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'

In [76]:
os.makedirs(model_path, exist_ok=True)

# Load fake news dataset from kaggle

In [77]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.drop(['id','title','author'],axis=1)
df=df.sample(frac=1)
df

,text,label
9965,These States Just Legalized Recreational Canna...,1
2363,"October 29, 2016 \nA man with no DIY experienc...",1
1619,Click Here To Learn More About Alexandra's Per...,1
8203,Solar-powered Pipe desalinates 1.5 billion gal...,1
12840,Email \nThe United States’ abstention on a res...,1
...,...,...
5387,"Ain’t you glad, Little Milton sang, that thing...",0
1754,"Sunday on CNN’s “GPS,” while discussing the ne...",0
6050,Teen Vogue asked young women gathered at CPAC ...,0
9362,Dems sue GOP over Trump's 'rigged' complaints ...,1


Split raw dataset to train set and validation set

In [78]:
train=df.iloc[:-5200]
valid=df.iloc[-5200:]

In [79]:
len(train),len(valid)

(15600, 5200)

Check data whether it is balanced or not

In [80]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 7752 samples
label: 1	has 7848 samples


In [81]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 2635 samples
label: 1	has 2565 samples


In [82]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [83]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [84]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'text'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'text'])


In [85]:
"""This function is from The Stanford Question Answering Dataset evaluate"""
def normalize(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  return white_space_fix(remove_articles(remove_punc(s))).strip()

In [86]:
train['text']=train['text'].map(normalize)

In [87]:
valid['text']=valid['text'].map(normalize)

In [88]:
train=train[train['text']!=""]
valid=valid[valid['text']!=""]

In [89]:
len(train),len(valid)

(15507, 5177)

In [90]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [91]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [92]:
tokenizer = AutoTokenizer.from_pretrained(name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [93]:
max_length=512
return_tensors='tf'

In [94]:
x_train=tokenizer(train['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [95]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [96]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [97]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [98]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [99]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [100]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [101]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [102]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

In [155]:
epochs=50
lr=5e-5
num_warmup_steps=5000

In [156]:
optimizer, schedule = create_optimizer(
    init_lr=lr, num_warmup_steps=num_warmup_steps, num_train_steps=num_training_samples*epochs
)

Load the pretrained model

In [157]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)
# model = TFAutoModelForSequenceClassification.from_config(config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [158]:
model.base_model_prefix

'bert'

Define train_step function for training

For token_type_ids

In [159]:
train_step_signature1=[
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature1)
def train_step1(input_ids, attention_mask,token_type_ids,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels,training = True)
    loss=outputs.loss
    probs=tf.nn.softmax(outputs.logits,axis=1)
    preds=tf.argmax(probs,axis=1)
    # loss = loss_fn(labels, probs)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss,preds

For not token_type_ids

In [160]:
train_step_signature2= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature2)
def train_step2(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,labels=labels,training = True)
    loss=outputs.loss
    probs=tf.nn.softmax(outputs.logits,axis=1)
    preds=tf.argmax(probs,axis=1)
    # loss = loss_fn(labels, probs)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss,preds

Defind test_step function for validation

In [161]:
def test_step(input_ids, attention_mask,token_type_ids,labels):
  
  if token_type_ids is None:
    outputs= model(input_ids=input_ids,attention_mask=attention_mask,labels=labels,training = False)
  else:
    outputs= model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels,training = False)
  loss=outputs.loss
  probs=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(probs,axis=1)
  # loss = loss_fn(labels, probs)

  return loss,preds

In [162]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Initializing the new checkpoint


In [163]:
threshold=1e-6

Train

In [164]:
def run_step(input_ids,attention_mask,token_type_ids,labels,session='train'):
  if session=='train':
    if model.base_model_prefix in ['distilbert']:
      return train_step2(input_ids,attention_mask,labels)
    else:
      return train_step1(input_ids,attention_mask,token_type_ids,labels)
  else:
    return test_step(input_ids,attention_mask,token_type_ids,labels)


In [165]:
def step(dataset,session='train'):
  
  loop = tqdm(dataset)
  loss=0
  y_true=[]
  y_pred=[]
  num_batches=0
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask'] if 'attention_mask' in batch else None
    token_type_ids=batch['token_type_ids'] if 'token_type_ids' in batch else None
    labels=batch['labels']

    _loss,preds=run_step(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,labels=labels,session=session)

    y_true.append(batch['labels'])
    y_pred.append(preds)
    _loss=tf.reduce_sum(_loss,axis=0).numpy()
    loss+=_loss
    num_batches+=1
    loop.set_postfix({'train_loss_per_batch':_loss})
    loop.update()

  y_true=tf.concat(y_true,axis=0)
  y_pred=tf.concat(y_pred,axis=0)
  acc=f1_score(y_true,y_pred)
  loss=loss/num_batches
  return acc,loss

In [ ]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_acc,train_loss=step(train_dataset)

  """Validation"""
  valid_acc,valid_loss=step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} train_acc :{train_acc} valid_loss: {valid_loss} valid_acc :{valid_acc}')
  if tf.abs(min_loss-valid_loss)>threshold and min_loss>valid_loss:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')
    min_loss=valid_loss

  0%|          | 0/1938 [00:00<?, ?it/s]

# Inferencing

Load test set

In [ ]:
test=pd.read_csv("/content/test.csv")

In [ ]:
submit=pd.read_csv("/content/submit.csv")
labels=tf.convert_to_tensor(submit['label'],dtype=tf.float32)

In [ ]:
test['text']=test['text'].astype(str).map(normalize)

In [ ]:
samples=test['text'].tolist()

In [ ]:
num_labels=len(submit['label'].unique())
num_labels

2

Load model

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [ ]:
model=TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.num_labels

2

In [ ]:
max_length=512
return_tensors='tf'
tokenizer = AutoTokenizer.from_pretrained(name)

Load the weights

In [ ]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Loading the latest checkpoint from /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-1


In [ ]:
y_pred=[]
loop = tqdm(range(len(samples)))
for idx,sample in enumerate(samples):
  inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)
  outputs=model(**inputs,training=False)
  prob=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(prob,axis=1)
  y_pred.append(preds)
  loop.update()


100%|██████████| 5200/5200 [15:35<00:00,  5.74it/s]

In [ ]:
y_pred=tf.convert_to_tensor(y_pred)

In [ ]:
y_pred=tf.reshape(y_pred,shape=(y_pred.shape[0],))
y_pred

<tf.Tensor: shape=(5200,), dtype=int64, numpy=array([0, 1, 1, ..., 0, 1, 0])>

In [ ]:
labels

<tf.Tensor: shape=(5200,), dtype=float32, numpy=array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)>

In [ ]:
f1_score(labels,y_pred,average='micro')

0.635576923076923